In [1]:
%%init_spark
launcher.packages= ["graphframes:graphframes:0.8.2-spark3.2-s_2.12"]

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._


Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.149:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1668542645146)
SparkSession available as 'spark'


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._


In [3]:
val vertexArray = Array(
  (1, "Alice", 28),
  (2, "Bob", 27),
  (3, "Charlie", 65),
  (4, "David", 42),
  (5, "Ed", 55),
  (6, "Fran", 50),
    (7, "Qing",27),
    (8, "Sarika",78),
    (9, "Olafson",17),
    (10, "Birgit",33)
)

val edgeArray = Array(
  (2, 1, 7),
  (1, 2, 13),
  (2, 4, 2),
  (3, 2, 4),
  (3, 6, 3),
  (4, 1, 1),
  (5, 2, 2),
  (5, 3, 8),
  (5, 6, 3),
    (7, 8, 14),
    (7, 9, 2),
    (8, 10, 8),
    (9, 10, 6)
)

val vertex_df = spark.createDataFrame(vertexArray).toDF("id","name","age")
val edge_df = spark.createDataFrame(edgeArray).toDF("src","dst","attr")

val g = GraphFrame(vertex_df, edge_df)

vertexArray: Array[(Int, String, Int)] = Array((1,Alice,28), (2,Bob,27), (3,Charlie,65), (4,David,42), (5,Ed,55), (6,Fran,50), (7,Qing,27), (8,Sarika,78), (9,Olafson,17), (10,Birgit,33))
edgeArray: Array[(Int, Int, Int)] = Array((2,1,7), (1,2,13), (2,4,2), (3,2,4), (3,6,3), (4,1,1), (5,2,2), (5,3,8), (5,6,3), (7,8,14), (7,9,2), (8,10,8), (9,10,6))
vertex_df: org.apache.spark.sql.DataFrame = [id: int, name: string ... 1 more field]
edge_df: org.apache.spark.sql.DataFrame = [src: int, dst: int ... 1 more field]
g: org.graphframes.GraphFrame = GraphFrame(v:[id: int, name: string ... 1 more field], e:[src: int, dst: int ... 1 more field])


In [5]:
g.inDegrees.printSchema

root
 |-- id: integer (nullable = false)
 |-- inDegree: integer (nullable = false)



In [17]:
g.inDegrees.rdd.map(r => (r.getInt(0),r.getInt(1))).collect

res13: Array[(Int, Int)] = Array((1,2), (4,1), (2,3), (6,2), (3,1), (9,1), (8,1), (10,2))


In [13]:
g.inDegrees.rdd.map(r => (r(0).toString.toInt,r.getInt(1))).collect

res9: Array[(Int, Int)] = Array((1,2), (4,1), (2,3), (6,2), (3,1), (9,1), (8,1), (10,2))


In [19]:
val in_deg = g.inDegrees.rdd.map(r => (r.getInt(0),r.getInt(1))).collect
val out_deg = g.outDegrees.rdd.map(r => (r.getInt(0),r.getInt(1))).collect
scala.util.Sorting.quickSort(out_deg) //Sorts an array in place
scala.util.Sorting.quickSort(in_deg) //Sorts an array in place
val result = in_deg sameElements out_deg //Returns "true" if the two arrays are identical in value
print(result)

false

in_deg: Array[(Int, Int)] = Array((1,2), (2,3), (3,1), (4,1), (6,2), (8,1), (9,1), (10,2))
out_deg: Array[(Int, Int)] = Array((1,1), (2,2), (3,2), (4,1), (5,3), (7,2), (8,1), (9,1))
result: Boolean = false


In [38]:
def make_undirected_graph(g: GraphFrame) = {
    val u_edge_df = g.find("(a)-[]->(b)")
        .select($"a.id".as("src"),$"b.id".as("dst"))
        .withColumn("swap",when(col("src")<col("dst"),col("dst")))
        .withColumn("dst",
                    when(col("swap").isNotNull,col("src"))
                    .otherwise(col("dst")))
        .withColumn("src",
                    when(col("swap").isNotNull,col("swap"))
                   .otherwise(col("src")))
        .drop(col("swap"))
        .distinct
    val u_vertices_df = g.vertices
    val u_g = GraphFrame(u_vertices_df,u_edge_df)    
    u_g
}

make_undirected_graph: (g: org.graphframes.GraphFrame)org.graphframes.GraphFrame


In [39]:
make_undirected_graph(g) //keep only one edge between any two vertices regardless of direction
    .edges //get the edges dataframe from this undirected graph
    .rdd //convert the edges dataframe into an rdd
    .map(r => Array(r.getInt(0),r.getInt(1))) //extract Array(src,dest) pairs for each edge
    .collect //convert into a Scala Array e.g. Array(Array(1,2),Array(2,4),Array(3,2))
    .flatten //flatten the array e.g., Array(1, 2, 2, 4, 3, 2)
    .groupBy(identity) //group the elements by each individual value (grouped by 1, 2, 4, 3)
    .mapValues(_.size) //return a Map object with each individual value as the key and the count as the value
                        //e.g., 1 -> 1, 2 -> 3, 4 -> 1, 3 -> 1

res29: scala.collection.immutable.Map[Int,Int] = Map(5 -> 3, 1 -> 2, 6 -> 2, 2 -> 4, 3 -> 3, 4 -> 2)


In [22]:
val u_g = make_undirected_graph(g)

u_g: org.graphframes.GraphFrame = GraphFrame(v:[id: int, name: string ... 1 more field], e:[src: int, dst: int])


In [24]:
u_g.edges.show

+---+---+
|src|dst|
+---+---+
|  2|  1|
|  4|  2|
|  3|  2|
|  6|  3|
|  4|  1|
|  5|  2|
|  5|  3|
|  6|  5|
|  9|  7|
|  8|  7|
| 10|  9|
| 10|  8|
+---+---+



In [25]:
g.edges.show

+---+---+----+
|src|dst|attr|
+---+---+----+
|  2|  1|   7|
|  1|  2|  13|
|  2|  4|   2|
|  3|  2|   4|
|  3|  6|   3|
|  4|  1|   1|
|  5|  2|   2|
|  5|  3|   8|
|  5|  6|   3|
|  7|  8|  14|
|  7|  9|   2|
|  8| 10|   8|
|  9| 10|   6|
+---+---+----+



In [31]:
val vertexArray = Array(
  (1, "Alice", 28),
  (2, "Bob", 27),
  (3, "Charlie", 65),
  (4, "David", 42),
  (5, "Ed", 55),
  (6, "Fran", 50)
)

val edgeArray = Array(
  (2, 1, 7),
  (1, 2, 13),
  (2, 4, 2),
  (3, 2, 4),
  (3, 6, 3),
  (4, 1, 1),
  (5, 2, 2),
  (5, 3, 8),
  (5, 6, 3)
)
val vertex_df = spark.createDataFrame(vertexArray).toDF("id","name","age")
val edge_df = spark.createDataFrame(edgeArray).toDF("src","dst","attr")
val graph = GraphFrame(vertex_df,edge_df)

vertexArray: Array[(Int, String, Int)] = Array((1,Alice,28), (2,Bob,27), (3,Charlie,65), (4,David,42), (5,Ed,55), (6,Fran,50))
edgeArray: Array[(Int, Int, Int)] = Array((2,1,7), (1,2,13), (2,4,2), (3,2,4), (3,6,3), (4,1,1), (5,2,2), (5,3,8), (5,6,3))
vertex_df: org.apache.spark.sql.DataFrame = [id: int, name: string ... 1 more field]
edge_df: org.apache.spark.sql.DataFrame = [src: int, dst: int ... 1 more field]
graph: org.graphframes.GraphFrame = GraphFrame(v:[id: int, name: string ... 1 more field], e:[src: int, dst: int ... 1 more field])


In [35]:
graph.filterEdges("attr>2").filterVertices("age < 50").edges.show

+---+---+----+
|src|dst|attr|
+---+---+----+
|  2|  1|   7|
|  1|  2|  13|
+---+---+----+



In [37]:
graph.filterEdges("attr>2").filterVertices("age < 50").vertices.show

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 28|
|  2|  Bob| 27|
|  4|David| 42|
+---+-----+---+

